In [1]:
from src.fitting.pygmo_bfes import *

In [3]:
import os
from datetime import datetime  
import tellurium as te
import numpy as np
import pygmo as pg
import matplotlib.pyplot as plt
import pandas as pd
import pickle

# load SBML model and get parameter labels
modelfile = os.getcwd()+'/models/240226MC_FULL.sbml' # filled nan values with mean for now
r = te.loadSBMLModel(modelfile)
parameter_values = r.getGlobalParameterValues()

# get data and metadata for experiment
with open('data/230623_Kinetics_DA/simulation_ready.pkl', 'rb') as f:
    simulation_ready = pickle.load(f)
    init_concentrations = simulation_ready['init_concentrations']   
    metadata = simulation_ready['metadata']
    measurements = simulation_ready['measurements']

metabolites = [s for s in r.getFloatingSpeciesIds() if 'EC' not in s]
model_species = pd.read_csv(os.getcwd()+'/src/frenda_brenda/Files/SpeciesBaseMechanisms.csv')

v_to_c_map = {model_species.loc[model_species['Label'] == s, 'Relative'].values[0]:s for s in metabolites}

# get all the parameters that can vary during the fitting
kinetics_labels = [p for p in r.getGlobalParameterIds() if np.any([k in p for k in ['Km','Kcat','Ki']])]
Gi_labels = [p for p in r.getGlobalParameterIds() if 'Gi' in p ]
metabolites_labels = [model_species.loc[model_species['Label'] == s, 'Relative'].values[0] for s in metabolites] #[s for s in r.getFloatingSpeciesIds() if 'EC' not in s]

parameter_labels = kinetics_labels + Gi_labels + metabolites_labels # + list(metabolites_labels)
parameter_labels.append('rel0') # what are good bounds for this?

# make upper and lower bounds
upper = np.array([r[i]*10 for i in kinetics_labels] + [1 for _ in Gi_labels] + [10 for _ in metabolites_labels] + [10])
lower = np.array([r[i]/10 for i in kinetics_labels] + [1/100 for _ in Gi_labels] + [1/10 for _ in metabolites_labels] + [1/10])

# define variables that don't change during fitting, but could be different across samples
# (dilution factor, biosynthesis metabolites, which enzymes are present)
variables = {}
for s in metadata['sample_labels']:
    variables[s] = {'dilution_factor':1/metadata['dilution_factor'][s]}
    variables[s] = {**variables[s] , **{p:1 for p in r.getGlobalParameterIds() if 'p_EC' in p}}
    variables[s] = {**variables[s] , **init_concentrations[s]}

from src.fitting.pygmo_problems import SBMLGlobalFit_Multi_Fly
myprob = SBMLGlobalFit_Multi_Fly(
    model = modelfile, 
    data = measurements, 
    parameter_labels=parameter_labels, 
    lower_bounds=lower, upper_bounds=upper, 
    metadata=metadata,
    variables=variables,
    scale=True)

prob = pg.problem(myprob)

In [4]:
import ipyparallel as ipp
client = ipp.Client(profile='cheme-ecfers')

client[:].apply_sync(os.chdir,'/mmfs1/gscratch/cheme/dalba/repos/ECFERS');

In [5]:
udbfe = pickleless_bfe(client_kwargs={'profile':'cheme-ecfers'}, view_kwargs = {}, prob = prob)

In [6]:
udbfe.init_view(client_kwargs={'profile':'cheme-ecfers'})

In [7]:
mybfe = pg.bfe(udbfe)

In [8]:
# pick parameters based on this thesis
# https://repository.up.ac.za/handle/2263/66233#:~:text=The%20thesis%20shows%20that%20the,empirical%20tests%20and%20theoretical%20reasoning.
algos = []
pops = []
replicates = 1
for i in range(replicates):
    a = pg.pso_gen(gen=10,omega=0.95, eta1=0.25, eta2=0.25, max_vel=0.01, variant=5, neighb_type=2, neighb_param=4)
    # a = pg.gaco(gen=10)
    a.set_bfe(mybfe)
    algo = pg.algorithm(a)
    algo.set_verbosity(1)
    algos.append(algo)
    pops.append(pg.population(prob, size = 0))

In [9]:
import faulthandler
faulthandler.enable()

total_nodes = 40*4
init_std = 0.17
xs = (np.expand_dims(myprob._scale(lower*10),-1)@np.ones((1,total_nodes))*(1+np.random.normal(0, init_std,(len(lower),total_nodes)))).T
fvs = udbfe(prob, xs.reshape(-1))

KeyboardInterrupt: 

In [ ]:
algos[0].evolve(pops[0])

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (695,) + inhomogeneous part.

In [19]:
udbfe._view.apply_sync(lambda x: x._setup(), ipp.Reference("prob"))

CompositeError: one or more exceptions raised in: <lambda>
[0:apply]AttributeError: 'pygmo.core.problem' object has no attribute '_setup'
[1:apply]AttributeError: 'pygmo.core.problem' object has no attribute '_setup'
[2:apply]AttributeError: 'pygmo.core.problem' object has no attribute '_setup'
[3:apply]AttributeError: 'pygmo.core.problem' object has no attribute '_setup'
.... 691 more exceptions ...

In [29]:
prob.fitness(lower)

NameError: name 'RoadRunner' is not defined